<a href="https://colab.research.google.com/github/heejininku/Class2022Spring/blob/main/2022S18_huggingface_gradio(5.18_%EC%88%98%EC%97%85%EC%A0%95%EB%A6%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr

어떤 모델은 gr.Interface.load("-"). launch() 이런식으로 적기만 하면 바로 실행되는 경우도 있음(ex. GPT2)_그러나 안되는 경우도 많음.

### [Image classification](https://huggingface.co/tasks/image-classification)

e.g. https://huggingface.co/google/vit-base-patch16-224 

제목만 가져왔는데 안되는 경우에는 How to use 가지고 와서 하기


How to use

In [ ]:
!pip install transformers

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image 
import requests #여기까진 필요한 걸 import해왔구나
#여기까지는 script의 형태 (함수 아님)

url = 'http://images.cocodataset.org/val2017/000000039769.jpg' 
#어떤 그림정보를 가지고 왔음(url주소자체가 그림임을 jpg를 통해 알 수 있음.)
image = Image.open(requests.get(url, stream=True).raw) #이게 나중에 입력으로 들어감
#url 이미지를 오픈해왔고, jpg가 숫자형태로 구성되어 있구나 이정도 이해하기

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
#from_pretrained- pretrained는 잘 학습된 행렬-모델을 가지고 오는 것
#위에는 feature_exctractor을 할 수 있는 모델이고 
#아래는 그것을 기반으로 Classification할 수 있는 모델이구나
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = feature_extractor(images=image, return_tensors="pt")
#(이미지는 숫자정보)인데 이 안에서 압축해서 핵심정보를 뽑아내는 것
outputs = model(**inputs) #outputs는 Classify된 것이 나오는 것.
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item() 
#그 중에서 확률이 젤 큰 거 class를 보여줘
print("Predicted class:", model.config.id2label[predicted_class_idx])

Predicted class: Egyptian cat


Demo in Gradio

이제 그라디오화 해야 함.

In [ ]:
def func (image): #앞서 설정한 image가 input으로 들어왔다고 생각하면 된다. 
#입출력으로 변형시켜줌
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
  
  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx] #변수로 받은 것
  return predicted_class

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")  
#이부분 없이 해도 가능은 한데, 이 셀의 의미는 우리 깃텁에 있는 이미지파일을 아예 예시로 주는 것.
#데모는 빠르게 해보라는 건데 사진 찾는데에 오래걸리면 안되잖아~

0

In [ ]:
gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch() 
#입출력의 유형를 정의해줌- 우리는 이미지를 입력, 텍스트를 출력으로 할거야
#예시는 리스트화해서 올려야 함.

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://19631.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f05575cb110>,
 'http://127.0.0.1:7862/',
 'https://19631.gradio.app')

여기 위까지를 5.18 강의에서 함.
(5.23에 다시 반복함)

### [Fill-Mask](https://huggingface.co/tasks/fill-mask)

e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.10731077939271927,
  'sequence': "hello i'm a fashion model.",
  'token': 4827,
  'token_str': 'fashion'},
 {'score': 0.0877445638179779,
  'sequence': "hello i'm a role model.",
  'token': 2535,
  'token_str': 'role'},
 {'score': 0.05338413640856743,
  'sequence': "hello i'm a new model.",
  'token': 2047,
  'token_str': 'new'},
 {'score': 0.046672213822603226,
  'sequence': "hello i'm a super model.",
  'token': 3565,
  'token_str': 'super'},
 {'score': 0.027095887809991837,
  'sequence': "hello i'm a fine model.",
  'token': 2986,
  'token_str': 'fine'}]

Demo in Gradio

In [ ]:
import pandas as pd
def func (text): #위에 text를 입력으로 받고
  unmasker = pipeline('fill-mask', model='bert-base-uncased')
  result = unmasker(text)
  df = pd.DataFrame(result) 
  #위에 셀의 값을 보면 list의 형태로 dictionary가 묶여있음.
  #pd.DataFrame하면 pandas 이용해서 dataframe화해버림 #구현된 거 보면 훨씬 보기좋음
  return df

In [ ]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]
#예시를 준 것

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe',examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://10991.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f0553a44650>,
 'http://127.0.0.1:7864/',
 'https://10991.gradio.app')

### [Token classification](https://huggingface.co/tasks/token-classification)

e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

#아래는 두개의 ai model(from_pretrained)을 불러오는 것 -일종의 전처리
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER") 
#각각의 tokens들이 어떤 class인지 구분

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
#nlp가 결국 모델임.
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example) #위에 있는 예시 문장을 입력으로 받음.
print(ner_results)

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


Demo in Gradio

In [ ]:
import pandas as pd #df하려면?(내 생각)
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result) 
  #위에 보면 리스트 안에 딕셔너리가 2개 존재하니까 df로 하는 게 좋을 것 
  #리스트로 되어 있을 때 그라디오에서 바로 output을 df 형태로 나타낼 수 있음.
  return df

In [ ]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [ ]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://28445.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f0557698a50>,
 'http://127.0.0.1:7865/',
 'https://28445.gradio.app')

### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
!pip install sentence_transformers 
#아래를 먼저했다가 에러메시지에 따라서 install해줌.

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"] 
#두 문장이 얼마나 비슷한지 계산을 해주는 것. 

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)
#그럼 embeddings가 무엇일까?-> 노트에 설명
#len(embeddings)는 2 , embeddings[0]하면 첫번째 리스트를 나타냄 
#len (embeddings[0]),[1]모두 384개의 길이

In [ ]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1]) 
#첫번째 문장, 두번째 문장에 대해 384개의 숫자와 cosine similarity를 나타내줌.
#추가 설명은 노트에
cosine_scores

tensor([[0.5726]])

Demo in Gradio

In [ ]:
def func (text1, text2): #내가 궁금한 점: text1, text2로 언제 위에서 지정이 된 거지? 위에는 그냥 sentences로 명명되어있지 않나?
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1]) 
  return cosine_scores

In [ ]:
examples = [["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]]

In [ ]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://19669.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f05309f10d0>,
 'http://127.0.0.1:7866/',
 'https://19669.gradio.app')